## Check Jupyter notebook is using pixi kernel

In [ ]:
# Cell 1: Check Python executable
import sys
print("Python executable:", sys.executable)

In [ ]:
# Cell 2: Check Python path
import sys
print("Python path:")
for path in sys.path:
    print(f"  {path}")

## Define Variables / Import MetaData

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from cartopy import crs as ccrs 
import cartopy.feature as cfeature
import pandas as pd
import hvplot.pandas
import xarray as xr
import hvplot.xarray
import geoviews.feature as gf
import datetime
import glob
import os
import pprint

In [ ]:
# Define misc variables 
# I can't get the config.py to work in jupyternotebook because it does not know where $NOBACKUP is
amer_filepath = '../../ameriflux-data/'
mic_filepath = '../../micasa-data/'
timedelta = 'DD'

In [ ]:
# Import site metadata csv
meta_file = amer_filepath + 'AmeriFlux-site-search-results-202410071335.tsv'
ameriflux_meta = pd.read_csv(meta_file, sep='\t')
fluxnet_meta = ameriflux_meta.loc[ameriflux_meta['AmeriFlux FLUXNET Data'] == 'Yes'] 

In [ ]:
total_sites = fluxnet_meta['Site ID'].to_list()
len(total_sites)

In [ ]:
# set map proj
proj=ccrs.PlateCarree()

In [ ]:
def get_single_match(pattern):
    matches = glob.glob(pattern)
    if len(matches) == 1:
        return matches[0]
    elif len(matches) == 0:
        raise ValueError(f"No matches found")
    else:
        raise ValueError(f"Multiple matches found: {matches}")

In [ ]:
def get_amer_match(amer_filepath, site_ID):
    match = get_single_match(amer_filepath + 'AMF_' + site_ID +
                            '_FLUXNET_SUBSET_*/AMF_' + site_ID +
                            '_FLUXNET_SUBSET_' + timedelta + '*.csv')
    return match

## 1. Develop script to dump all variables requested into one preprocessed file

In [ ]:
site_ID = 'AR-TF1'
# Define micasa variables wanted
micasa_var_list = ['NEE', 'NPP']
# Extract site lat/lon
site_lat = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Latitude (degrees)'].values
site_lon = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Longitude (degrees)'].values

In [ ]:
# Import fluxnet
fluxnet_sel = pd.read_csv(get_amer_match(amer_filepath, site_ID))

In [ ]:
fluxnet_sel_dates = fluxnet_sel.loc[:,['TIMESTAMP']].copy()
fluxnet_sel_dates['TIMESTAMP'] = pd.to_datetime(fluxnet_sel_dates['TIMESTAMP'],format='%Y%m%d')
fluxnet_sel_dates = fluxnet_sel_dates.set_index('TIMESTAMP')
time = fluxnet_sel_dates.index
dates_unique = list({dt.date() for dt in time})
dates_unique.sort()
dates_unique[:5]

In [ ]:
# Import micasa
# Test some random files
path_test_mon = glob.glob(mic_filepath + 'daily/2016/01/MiCASA_v1_flux_x3600_y1800_daily_201601*.nc4')
path_sorted_mon = sorted(path_test_mon)

path_test_yr = glob.glob(mic_filepath + 'daily/2016/??/MiCASA_v1_flux_x3600_y1800_daily_20??????.nc4')
path_sorted_yr = sorted(path_test_yr)

path_test_all = glob.glob(mic_filepath + 'daily/20??/??/MiCASA_v1_flux_x3600_y1800_daily_20??????.nc4')
path_sorted_all = sorted(path_test_all)

In [ ]:
print(path_sorted_mon[0], len(path_sorted_mon), len(path_sorted_yr), len(path_sorted_all), sep=',  ')

In [ ]:
%%time
# ds = xr.open_mfdataset(path_test_mon)
# ds = xr.open_mfdataset(path_sorted_mon)[micasa_var_list] 
ds = xr.open_mfdataset(path_sorted_mon, 
                       engine = 'h5netcdf',
                       # combine='nested', concat_dim='time',
                      )[micasa_var_list]
# ds = xr.open_mfdataset(path_sorted_mon, parallel=True)[micasa_var_list] #Test this when I can get the dashboard to work
ds

# Note: 
# Not clear how fast combine=nested, concat_dim='time' improves speed over default combine='by-coords'
# Adding in just relevant variables seems to shorten time several seconds for 1 month

In [ ]:
# IMO this is all slow and i should virtualize... because then I can reuse the virtualized store for across all sites, since there are year crossovers?
# Or perhaps I should load all the micasa data at once and subset from there? That would avoid opening some files multiple times
# How long does it take to open the micasa data? ~10 seconds for 1 month, 1:30 for 1 year, too long to run all

In [ ]:
# %%time
ds_subset = ds.sel(lon=site_lon, lat=site_lat, method='nearest')
ds_subset

In [ ]:
# ds_out = pd.DataFrame()
# with xr.open_mfdataset(path_test_mon)[micasa_var_list] as ds:
#     # Select grid closest to selected site
#     ds_subset = ds.sel(lon=site_lon, lat=site_lat, method='nearest')
#     # print(ds_subset)
    
#     # Prep data for writing to csv
#     ds_subset = ds_subset.squeeze(dim=['lat','lon'],drop=True)
#     # print(ds_subset)
    
#     # Output a single file for each site with all variables
#     for micasa_var in micasa_var_list:
#         ds_out[micasa_var] = ds_subset[micasa_var].to_dataframe()
#         ds_out.rename(columns={micasa_var: f'MiCASA {micasa_var} ({ds_subset[micasa_var].units})'}, inplace=True)

## 2. Debug Preprocessing

### 2a. Check sites that aren't preprocessed or plotted

In [ ]:
# Get list of files and create dataframe with truncated filenames
plots_list = !ls ../plotting/plots
plotted_sites = list([filename.split('_')[0] for filename in plots_list])

In [ ]:
len(plotted_sites)

In [ ]:
# Find missing sites that are not plotted:
missing_plots = [item for item in total_sites
                if item not in plotted_sites]

missing_df = pd.DataFrame(missing_plots, columns=['Missing from Plots'])
missing_df

In [ ]:
# What about intermediates?
intermediates_list = !ls ../preprocessing/intermediates/
processed_sites = list([filename.split('_')[0] for filename in intermediates_list])

In [ ]:
missing_processed = [item for item in total_sites
                if item not in processed_sites]
missing_df2 = pd.DataFrame(missing_processed, columns=['Missing from Intermediates'])
missing_df2

##### No longer missing! Bugs fixed

### 2b. Debugging pre 2001 FLUXNET data

In [ ]:
def get_single_match(pattern):
  matches = glob.glob(pattern)
  if len(matches) == 1:
      return matches[0]
  elif len(matches) == 0:
      raise ValueError(f"No matches found for: {pattern}")
  else:
      raise ValueError(f"Multiple matches found: {matches}")

timedelta = 'DD'
micasa_var_list = ['NEE', 'NPP']

#Import list of fluxnet sites
meta_file = amer_filepath + 'AmeriFlux-site-search-results-202410071335.tsv'
ameriflux_meta = pd.read_csv(meta_file, sep='\t')
fluxnet_meta = ameriflux_meta.loc[ameriflux_meta['AmeriFlux FLUXNET Data'] == 'Yes'] #use FLUXNET only
fluxnet_list = fluxnet_meta['Site ID'].to_list()

In [ ]:
# see the dates for the erroring sites
for site_ID in fluxnet_list:
    # Open site data and access time indices
    site_file = get_single_match(amer_filepath + 'AMF_' + site_ID +
                              '_FLUXNET_SUBSET_*/AMF_' + site_ID +
                              '_FLUXNET_SUBSET_' + timedelta + '*.csv')
    fluxnet_sel = pd.read_csv(site_file)
    
    # select subset of columns + convert to datetime objects
    fluxnet_sel_dates = fluxnet_sel.loc[:,['TIMESTAMP']].copy()
    fluxnet_sel_dates['TIMESTAMP'] = pd.to_datetime(fluxnet_sel_dates['TIMESTAMP'],format='%Y%m%d')
    fluxnet_sel_dates = fluxnet_sel_dates.set_index('TIMESTAMP')
    
    # Create a list of unique dates from the site
    time = fluxnet_sel_dates.index
    dates_unique = list({dt.date() for dt in time})
    dates_unique.sort()

    
    # Extract micasa data
    data_path = mic_filepath + 'daily/'
    
    path_list = []
    for date in dates_unique:
        f_year = str(date.year)
        f_month = f"{date.month:02}"
        filename = 'MiCASA_v1_flux_*' + date.strftime('%Y%m%d') + '.nc4'
        # print(os.path.join(data_path,f_year,f_month,filename))
        # match = get_single_match(os.path.join(data_path,f_year,f_month,filename))
        # print(match)
        # sys.exit()
        try:
            get_single_match(os.path.join(data_path,f_year,f_month,filename))
        except ValueError as e:
            print(f"{site_ID} has Fluxnet data for: {dates_unique[0]} to {dates_unique[-1]}")
            break

In [ ]:
# try to skip the error for one site
site_ID = "CA-Ca1"

# Open site data and access time indices
site_file = get_single_match(amer_filepath + 'AMF_' + site_ID +
                          '_FLUXNET_SUBSET_*/AMF_' + site_ID +
                          '_FLUXNET_SUBSET_' + timedelta + '*.csv')
fluxnet_sel = pd.read_csv(site_file)

# select subset of columns + convert to datetime objects
fluxnet_sel_dates = fluxnet_sel.loc[:,['TIMESTAMP']].copy()
fluxnet_sel_dates['TIMESTAMP'] = pd.to_datetime(fluxnet_sel_dates['TIMESTAMP'],format='%Y%m%d')
fluxnet_sel_dates = fluxnet_sel_dates.set_index('TIMESTAMP')

# Create a list of unique dates from the site
time = fluxnet_sel_dates.index
dates_unique = list({dt.date() for dt in time})
dates_unique.sort()


# Extract micasa data
data_path = mic_filepath + 'daily/'

path_list = []
for date in dates_unique:
    f_year = str(date.year)
    f_month = f"{date.month:02}"
    filename = 'MiCASA_v1_flux_*' + date.strftime('%Y%m%d') + '.nc4'
    # print(os.path.join(data_path,f_year,f_month,filename)
    
    try:
        filepath = get_single_match(os.path.join(data_path,f_year,f_month,filename))
        path_list.append(filepath)
    except ValueError as e:
        continue

In [ ]:
print(dates_unique[0])
path_list[0]

## Debug Land Mask

In [ ]:
import seaborn as sns

### FluxNet Land Type / Tables of Site lat/lon for Brad

In [ ]:
summary_table = fluxnet_meta[['Site ID','Latitude (degrees)','Longitude (degrees)', 'Vegetation Abbreviation (IGBP)', 'Vegetation Description (IGBP)', 'Climate Class Abbreviation (Koeppen)', 'Climate Class Description (Koeppen)']]
summary_table

### Plot AmeriFlux sites

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6), subplot_kw= {'projection': proj})
ax.add_feature(cfeature.COASTLINE,zorder=0)
sns.scatterplot(x='Longitude (degrees)', y='Latitude (degrees)', data=summary_table, hue='Vegetation Abbreviation (IGBP)', ax=ax)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# Plot vegetation abbreviation?
veg_dict = dict(zip(summary_table['Vegetation Abbreviation (IGBP)'].unique(),summary_table['Vegetation Description (IGBP)'].unique()))

In [ ]:
pprint.pprint(veg_dict)

### MiCASA Land Mask

In [ ]:
!ls ../../landmask

In [ ]:
# Import landmask file for 2001
landmask_filepath = '../../landmask/'
year = str(2001)
ds = xr.open_dataset(landmask_filepath + 'MiCASA_v1_cover_x3600_y1800_yearly_' + year + '.nc4')

In [ ]:
ds.ftype

In [ ]:
ds_water = ds.ftype.sel(type=17)

In [ ]:
ds_water.plot()

In [ ]:
# ds.ftype.hvplot(x='lat',y='lon', 
#                 crs=proj,
#                size=150)

## Coastal Site Issue

In [ ]:
site_ID = 'US-KS3' # example site that is showing up weird
# Extract site lat/lon
site_lat = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Latitude (degrees)'].values[0]
site_lon = ameriflux_meta.loc[ameriflux_meta['Site ID'] == site_ID, 'Longitude (degrees)'].values[0]
print(site_lat, site_lon)

In [ ]:
# Approx location of site
ax = plt.subplot(projection=proj,frameon=False)
if site_lat >= 20:
    # North America extents
    min_lon, max_lon = -170, -57
    min_lat, max_lat = 25, 74

else:
    # South America extents
    min_lon, max_lon = -90, -30
    min_lat, max_lat = -60, 12

ax.axis('off')
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
ax.coastlines()

ax.scatter(site_lon,site_lat,
       marker='*', 
       s=500,
       color='yellow',
       edgecolor='black',
               zorder=3)

In [ ]:
# Subset data for plotting
min_lon, max_lon = site_lon-5, site_lon+5
min_lat, max_lat = site_lat-2, site_lat+2

#### Single file

In [ ]:
mult_path_test = glob.glob(mic_filepath + 
                           '/daily/2016/01/MiCASA_v1_flux*.nc4')

In [ ]:
ds = xr.open_dataset(mult_path_test[0])['NEE']
ds_subset = ds.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon,max_lon)).isel(time=0)
ds_subset

#### Multifile

In [ ]:
import h5netcdf
import dask
dask.config.set({'array.slicing.split_large_chunks': True})

In [ ]:
mult_ds = xr.open_mfdataset(
    mult_path_test, 
    engine='h5netcdf',
    parallel=True,  # Enable parallel reading
    chunks='auto'   # Let dask choose chunk sizes
)['NEE']
mult_ds

In [ ]:
ds_subset = mult_ds.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon,max_lon))
ds_subset.min().load(), ds_subset.max().load()

In [ ]:
mesh_plot = ds_subset.hvplot(x="lon", y="lat",
                      cmap='RdBu_r',
                  clim=(-2e-9,3e-8),
                      # crs = proj,
                      # rasterize=True,
                 frame_width = 500,
                 # widget_location='bottom'
                     )
mesh_plot

In [ ]:
ds_sel = ds_subset.sel(lon=[site_lon], lat=[site_lat], method='nearest')

In [ ]:
point = ds_sel.hvplot.points('lon', 'lat',
                             color='yellow',size=150,
                              # crs=proj,
                              # geo=True
                             # widget_location='bottom'
                            )
type(point)

In [ ]:
type(mesh_plot), type(point)

In [ ]:
mesh_plot * point

In [ ]:
## Let's look at some of the other sites that plot zero, where they are:
ID_list = ['US-EDN' , 'US-HB1', 'US-KS3']# example site that is showing up weird
filtered_df = fluxnet_meta[fluxnet_meta['Site ID'].isin(ID_list)]
filtered_df[['Site ID','Latitude (degrees)','Longitude (degrees)',]]

In [ ]:
# Create df with lat/lons
site_subset = ['Site ID', 
               'Longitude (degrees)',
                'Latitude (degrees)',
               ]
df_meta = fluxnet_meta[site_subset]
df_meta.set_index('Site ID')

In [ ]:
# Import and merge results
results = pd.read_csv('../analysis/results.csv',index_col='SiteID')
results

In [ ]:
df = df_meta.join(results, on='Site ID')
df

In [ ]:
ds

In [ ]:
# Make transparency colormap:
cmap = plt.cm.autumn_r
cmap

In [ ]:
cmap(np.arange(cmap.N)).shape

In [ ]:
cmap(1)

In [ ]:
my_cmap = cmap(np.arange(cmap.N))
my_cmap[:, -1] = np.linspace(0, 1, cmap.N)
my_cmap[-1]

In [ ]:
my_cmap = ListedColormap(my_cmap)
my_cmap

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12, 10), subplot_kw={'projection': proj}, constrained_layout=True);
fig.suptitle('MiCASA, FluxNet Sites Root Mean Squared Error (RSME)', y=0.76)
values =['NEE_RSME', 'NPP_RSME']
for ax,val in zip(axs, values):
    ax.add_feature(cfeature.COASTLINE,zorder=0)
    plot = ds.plot.scatter(x="lon", y="lat",ax=ax,
                           
                           markersize=val, edgecolor='none',add_legend=False,
                           
                            norm=colors.LogNorm(), 
                            # norm=colors.LogNorm(vmin=ds[val].min(), vmax=ds[val].max()),
                           hue=val,
                           cmap=my_cmap,
                           # cmap='cool',
                           # cmap='autumn_r',
                           add_colorbar=False
                          )
    
    cbar = fig.colorbar(plot, ax=ax, shrink=0.9, label=val[4:], orientation='horizontal')
    ax.set_title(val[:3])
plt.show()

In [ ]:
%matplotlib ipympl
ds.plot.scatter(x="lon", y="lat")

In [ ]:
df.head()

In [ ]:
min_lon, max_lon = df["Longitude (degrees)"].min(), df["Longitude (degrees)"].max()
min_lat, max_lat = df["Latitude (degrees)"].min(), df["Latitude (degrees)"].max()

print(min_lon, max_lon)
print(min_lat, max_lat)

In [ ]:
my_cmap

In [ ]:
plot = df.hvplot.points(x="Longitude (degrees)", 
                        y="Latitude (degrees)",
                        geo=True, 
                        crs=ccrs.PlateCarree(),
                        # projection=ccrs.PlateCarree(), # Doesn't work with tiles

                         #Custom cmap with transparency won't show up in bokeh
                        c=values[0],
                        logz=True,
                        cmap=my_cmap,
                        clabel=f'{values[0]}',

                         size=45,
                         # Size values don't scale logarithmically
                        # s=values[0],
                        # scale=4500,
                         # color='red',
                        
                        tiles=True,
                        tiles_opts={'alpha': 0.4},
                        # tiles=xyz.Esri.WorldGrayCanvas,


                        hover_cols=['Site ID'],

                        # width=700, height=500,
                        # xlim=(-165, -40),   # longitude range
                        # ylim=(-60, 75),     # latitude range
                        frame_width=800,
                        frame_height=500
                                           )
plot

In [ ]:
plot2 * gf.coastline